# *Contrastive Learning* pipeline

In [1]:
%load_ext autoreload
%autoreload 2
import datasets_handler
import evaluation_metrics
from zeroberto import ZeroBERTo
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import gc
import pandas as pd

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:169: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


In [2]:
which_dataset = 'folhauol' 
max_inferences = 10000

hyp_template = "Esse texto está relacionado a {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())

In [3]:
zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual',
    'setfit_model' : 'sentence-transformers/stsb-xlm-r-multilingual',# 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2'
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"fewshot",
    'labeling_method':zeroshot_method,
    'max_inferences':max_inferences,
    'classes':classes_list,
    'template': hyp_template,
    'random_state':422,
    'top_n': 8,
    'training_examples': 8,
    "batch_size" : 8,
    "num_pairs" : 4,
    "num_epochs" : 1,
    'st_train_epochs': 10,
    'st_train_batch_size': 20,
    # Unfreeze the head and freeze the body -> head-only training
    # 'keep_body_frozen_setfit': True,
    # Unfreeze the head and unfreeze the body -> end-to-end training
    'keep_body_frozen_setfit': False,
}

data_to_label = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()


In [4]:

model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
                  

AttributeError: 'SentenceTransformer' object has no attribute 'split'

In [ ]:
model.embeddingModel

In [15]:
data_to_label

,Unnamed: 0,title,date,class,subcategory,link,data,text,len
6,8,Supremo nega pedido para Senado analisar impea...,2017-09-10,Poder e Política no Brasil,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,Supremo nega pedido para Senado analisar impea...,Supremo nega pedido para Senado analisar impea...,1539
22,27,"Plano de Energia Limpa de Obama será revogado,...",2017-09-10,Meio Ambiente,NaN,http://www1.folha.uol.com.br/ambiente/2017/10/...,"Plano de Energia Limpa de Obama será revogado,...","Plano de Energia Limpa de Obama será revogado,...",3183
24,29,MPF pede que prisão de Nuzman seja prolongada ...,2017-09-10,Esporte,NaN,http://www1.folha.uol.com.br/esporte/2017/10/1...,MPF pede que prisão de Nuzman seja prolongada ...,MPF pede que prisão de Nuzman seja prolongada ...,2464
32,42,Nuzman tentou pagar advogado criminal com recu...,2017-09-10,Esporte,NaN,http://www1.folha.uol.com.br/esporte/2017/10/1...,Nuzman tentou pagar advogado criminal com recu...,Nuzman tentou pagar advogado criminal com recu...,2215
40,54,Adiada permissão para pagar boleto vencido até...,2017-09-10,Mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...,Adiada permissão para pagar boleto vencido até...,Adiada permissão para pagar boleto vencido até...,2303
...,...,...,...,...,...,...,...,...,...
92202,166965,Veja as manchetes dos principais jornais desta...,2015-01-01,Poder e Política no Brasil,NaN,http://www1.folha.uol.com.br/poder/2015/01/156...,Veja as manchetes dos principais jornais desta...,Veja as manchetes dos principais jornais desta...,908
92211,167000,"Em Miami, brasileiros gastam mais com compras ...",2015-01-01,Mercado,NaN,http://www1.folha.uol.com.br/mercado/2015/01/1...,"Em Miami, brasileiros gastam mais com compras ...","Em Miami, brasileiros gastam mais com compras ...",972
92228,167033,Novo chanceler faz parte de grupo ligado a Amorim,2015-01-01,Poder e Política no Brasil,NaN,http://www1.folha.uol.com.br/poder/2015/01/156...,Novo chanceler faz parte de grupo ligado a Amo...,Novo chanceler faz parte de grupo ligado a Amo...,1318
92231,167040,Economista assume governo de PE e promete 'mod...,2015-01-01,Poder e Política no Brasil,NaN,http://www1.folha.uol.com.br/poder/2015/01/156...,Economista assume governo de PE e promete 'mod...,Economista assume governo de PE e promete 'mod...,2683


In [4]:
# setfit_model = SetFitModel.from_pretrained('ricardo-filho/bert-base-portuguese-cased-nli-assin-2')
model.contrastiveModel  = SetFitModel.from_pretrained('ricardo-filho/bert-base-portuguese-cased-nli-assin-2', use_differentiable_head=True,
                                        head_params={"out_features":len(classes_list)})

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [14]:
train_dataframe = datasets_handler.splitDataset(model.labeling_dataset,model.config) ### df_test will not be used

# train_data['class_code'] = train_data['class_code'].astype(int)
# train_data['text'] = train_data['text'].astype(str)
train_data = datasets_handler.buildDatasetDict(train_dataframe)


KeyError: 'class_code'

In [13]:
### build trainer for contrastive learning
model.buildTrainer(train_data)

### train
# model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
model.getPredictions()
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese

setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred.cpu(), model.trainer.eval_dataset["class_code"])
print(setfit_all_metrics)

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics)
print(model.config)

KeyError: "['class_code'] not in index"

In [7]:
model.y_pred

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:162: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,

In [9]:
train_data['class_code']

[4,
 4,
 8,
 8,
 11,
 6,
 8,
 8,
 7,
 6,
 8,
 4,
 6,
 6,
 7,
 4,
 6,
 7,
 4,
 7,
 5,
 8,
 8,
 8,
 5,
 6,
 6,
 6,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 11,
 3,
 9,
 2,
 11,
 11,
 0,
 3,
 0,
 0,
 0,
 0,
 9,
 0,
 2,
 2,
 11,
 9,
 3,
 2,
 10,
 11,
 10,
 11,
 2,
 9,
 11,
 9,
 9,
 9,
 9,
 2,
 10,
 2,
 10,
 0,
 1,
 10,
 1,
 10,
 0,
 10,
 10,
 2,
 3,
 5,
 5,
 1,
 1,
 3,
 3,
 1,
 1,
 1,
 1]

In [16]:
model.trainer.evaluate()

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:163: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  all_sentences = np.array(docs.apply(lambda x : re.sub(r'(\d+)(\.)(\d+)',r"\1"+r"\3", x)).str.split("."))
0it [00:00, ?it/s]
Applying column mapping to evaluation dataset
***** Running evaluation *****


{'accuracy': 0.253}